In [2]:
!pip install einops

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for einops from https://files.pythonhosted.org/packages/44/5a/f0b9ad6c0a9017e62d4735daaeb11ba3b6c009d69a26141b258cd37b5588/einops-0.8.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 440.9 kB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import sys

import support_functions

sys.path.append(os.path.join(os.path.abspath(os.getcwd()), "PEGNN"))
import json
import time
import myconfig_trsage as myconfig
import solver_trsage as solver
import Dataloader_trsage as dl
from datetime import datetime
import torch

def make_dir(path):
    try:
        os.mkdir(path)
    except:
        pass


# rebuild the folder missed?
def build_folder_and_clean(path):
    check = os.path.exists(path)
    if check:
        pass
    else:
        os.makedirs(path)


def train(job_id, settings):
    result_sheet = []

    print("Start training...")
    list_total, list_err = solver.training(settings=settings, job_id=job_id)
    print("Start evaluation...")
    best_err, r_squared = solver.evaluate(settings=settings, job_id=job_id)

    result_sheet.append([list_total, list_err, best_err, r_squared])

    # collect wandb result into file
    rtn = {
        "best_err": sum(result_sheet[0][2]) / len(result_sheet[0][2]),
        "r_squared": sum(result_sheet[0][3]) / len(result_sheet[0][3]),
        "list_total_0": result_sheet[0][0],
        "list_err_0": result_sheet[0][1],
    }

    json_dump = json.dumps(rtn)
    with open(settings['agent_dir'] + f'/{job_id}.rtn', 'w') as fresult:
        fresult.write(json_dump)



# RuntimeError: mat1 and mat2 shapes cannot be multiplied (2974x42 and 46x256)
# problem for number of the dataset_size since i change the size into minimal size
# but if this problem occurs in ssh server then means all right
if __name__ == '__main__':
    job_id = '000002'

    print('Init...')

    settings = {
        'agent_id': '00001',
        'agent_dir': './logs',
        'origin_path': './Dataset_res250_reg4c/',

        # debug mode=>data_set
        'debug': True,
        'bp': False,

        # full_batch->batch->accumulation_steps double
        'batch': 16,
        'accumulation_steps': 128 // 16,
        'test_batch': 0,

        'es_mindelta': 0.5,

        # 'num_features_in': 14,
        'num_features_in': 2,

        'num_features_out': 1,
        'emb_hidden_dim': 256,
        
        'k': 20,
        'conv_dim': 256,


        # only in this case delete in slurm
        'seed': 1,
        'holdout': 3,

        
        'model': 'trsage5',
        'fold': 4,
        'lowest_rank': 1,

        'hp_marker': 'tuned',
        'nn_length': 3,
        'nn_hidden_dim': 32,
        'dropout_rate': 0.1,

        # for transformer
        'd_model': 32,
        'num_head': 2,

        'feedforward_dim': 128,
        'dropout': 0.1,
        'num_layers_a': 2,
        'env_features_in': 11,

        
        'transformer_dec_output': 32,
        'emb_dim': 32,
        'epoch': 8,
        'es_endure': 5,
        'nn_lr': 1e-5,
        
        'trans_lr': 1e-5,

        'embedding_dim': 256,

    }

    # build working folder
    dt_string = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    coffer_slot = myconfig.coffer_path + str(job_id) + '/'

    # missed
    make_dir(coffer_slot)
    build_folder_and_clean(coffer_slot)
    settings['coffer_slot'] = coffer_slot
    settings['tgt_op'] = 'mcpm10'

    train(job_id, settings)

Init...
Start training...
{
  "agent_id": "00001",
  "agent_dir": "./logs",
  "origin_path": "./Dataset_res250_reg4c/",
  "debug": true,
  "bp": false,
  "batch": 16,
  "accumulation_steps": 8,
  "test_batch": 0,
  "es_mindelta": 0.5,
  "num_features_in": 2,
  "num_features_out": 1,
  "emb_hidden_dim": 256,
  "k": 20,
  "conv_dim": 256,
  "seed": 1,
  "holdout": 3,
  "model": "trsage5",
  "fold": 4,
  "lowest_rank": 1,
  "hp_marker": "tuned",
  "nn_length": 3,
  "nn_hidden_dim": 32,
  "dropout_rate": 0.1,
  "d_model": 32,
  "num_head": 2,
  "feedforward_dim": 128,
  "dropout": 0.1,
  "num_layers_a": 2,
  "env_features_in": 11,
  "transformer_dec_output": 32,
  "emb_dim": 32,
  "epoch": 8,
  "es_endure": 5,
  "nn_lr": 1e-05,
  "trans_lr": 1e-05,
  "embedding_dim": 256,
  "coffer_slot": "./coffer_trsage_gee/000002/",
  "tgt_op": "mcpm10"
}
Working on CPU
Length of df dict: 100
Length of call list: 4352
Length of df dict: 100
Length of call list: 3840
INTP_Model(
  (fc_mu_d): Linear(in_fe

/home/kit/stud/uqqww/.local/lib/python3.9/site-packages/mpl_scatter_density/generic_density_artist.py:77: RuntimeWarning: All-NaN slice encountered
  vmin = self._density_vmin(array)
/home/kit/stud/uqqww/.local/lib/python3.9/site-packages/mpl_scatter_density/generic_density_artist.py:82: RuntimeWarning: All-NaN slice encountered
  vmax = self._density_vmax(array)


Current epoch: 1


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>